In [ ]:
import requests
import pandas as pd
import time

# Spotify API credentials
CLIENT_ID = "99d8401365114e2da8eb836284bc8be0"
CLIENT_SECRET = "91c15af888db425083ed55e374fadff6"

# Spotify API endpoints
TOKEN_URL = "https://accounts.spotify.com/api/token"
BASE_URL = "https://api.spotify.com/v1"

def get_access_token(client_id, client_secret):
    """
    Fetch an access token using client credentials.
    """
    data = {"grant_type": "client_credentials"}
    auth_response = requests.post(
        TOKEN_URL,
        data=data,
        auth=(client_id, client_secret)
    )
    response_data = auth_response.json()
    if auth_response.status_code == 200:
        return response_data["access_token"]
    else:
        raise Exception(f"Error getting access token: {response_data}")

def get_artist_genres(access_token, artist_id):
    """
    Get genres for a given artist.
    """
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(f"{BASE_URL}/artists/{artist_id}", headers=headers)
    if response.status_code == 200:
        artist_data = response.json()
        return artist_data.get("genres", [])
    elif response.status_code == 404:
        print(f"Artist {artist_id} not found. Skipping.")
        return []
    elif response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 1))
        print(f"Rate limit exceeded. Retrying after {retry_after} seconds.")
        time.sleep(retry_after)
        return get_artist_genres(access_token, artist_id)
    else:
        print(f"Unexpected error fetching artist {artist_id}: {response.status_code} - {response.text}")
        return []

def append_genres_to_dataset(dataset, client_id, client_secret):
    """
    Append genres to the dataset using Spotify API.
    """
    # Fetch access token
    access_token = get_access_token(client_id, client_secret)
    
    # Add a new column for genres
    genres = []

    # Iterate over each artist_id in the dataset
    for artist_id in dataset["artist_id"]:
        if pd.isna(artist_id):
            genres.append("")
            continue

        # Fetch genres for the artist
        artist_genres = get_artist_genres(access_token, artist_id)
        genres.append(", ".join(artist_genres))
    
    # Add genres to the dataframe
    dataset["genres"] = genres
    return dataset

# Load the dataset
file_path = "https://raw.githubusercontent.com/orrefo/Music/refs/heads/main/all.csv"
dataset = pd.read_csv(file_path)

# Append genres to the dataset
dataset_with_genres = append_genres_to_dataset(dataset, CLIENT_ID, CLIENT_SECRET)

# Save the updated dataset
output_path = "all_with_genres.csv"
dataset_with_genres.to_csv(output_path, index=False)
print(f"Updated dataset saved to {output_path}")


Rate limit exceeded. Retrying after 19952 seconds.
